In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
def getYFromBGR(imgBGR):
    imgYCrCb = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2YCrCb)
    return imgYCrCb[:,:,0]

def putYToBGR(imgY, imgBGR):
    imgYCrCb = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2YCrCb)
    imgY = np.abs(imgY).astype(np.uint8)
    imgYCrCb[:,:,0] = imgY
    imgBGR2 = cv2.cvtColor(imgYCrCb, cv2.COLOR_YCrCb2BGR)
    return imgBGR2

In [ ]:
def calcBE(W1, W2):
    return np.sum(np.abs(np.abs(W1) - np.abs(W2)))

In [ ]:
N_SCRAMBLE = 13

def scramble(logo, n=N_SCRAMBLE):
    if(n==0): return logo
    (h, w) = logo.shape
    newLogo = np.zeros(logo.shape)
    for y in range(h):
        for x in range(w):
            val = logo[y, x]
            newLogo[(y + x) % h, (y + 2 * x) % w] = val
    return scramble(newLogo, n-1)

def unscramble(logo, n=N_SCRAMBLE):
    if(n==0): return logo
    (h, w) = logo.shape
    newLogo = np.zeros(logo.shape)
    for y in range(h):
        for x in range(w):
            val = logo[y, x]
            newLogo[(2 * y - x) % h, (- y + x) % w] = val
    return unscramble(newLogo, n-1)

def scramble1D(array, n=N_SCRAMBLE):
    if(n==0): return array
    (l, ) = array.shape
    f1 = np.array([], dtype=bool)
    f2 = np.array([], dtype=bool)
    for i in range(l):
        if(i % 2 == 0):
            f1 = np.append(f1, True)
            f2 = np.append(f2, False)
        else:
            f1 = np.append(f1, False)
            f2 = np.append(f2, True)
    arr1 = array[f1]
    arr2 = array[f2]
    result = np.append(arr1, arr2)
    result = np.roll(result, 1)
    del f1, f2, arr1, arr2
    return scramble1D(result, n-1)

def unscramble1D(array, n=N_SCRAMBLE):
    if(n==0): return array
    array = np.roll(array, -1)
    (l, ) = array.shape
    l2 = np.ceil(l / 2)
    arr1 = array[0:int(l2)]
    arr2 = array[int(l2):]
    result = np.array([])
    for i in range(l):
        if(i % 2 == 0):
            result = np.append(result, arr1[int(i/2)])
        else:
            result = np.append(result, arr2[int(i/2)])
    del arr1, arr2
    return unscramble1D(result, n-1)

In [1]:
SN = 0
DN = 0
ST = 2
PX = 100

def watermarkEmbed(logo, p):
    logo_sc = scramble(logo)
    p2 = 2 * p + 1
    count = 0
    for n in range(1 + SN, p):
        for m in range(p2):
            l = m - p
            if(l % 4 != 0 and np.abs(l) + n >= DN and np.abs(l) + n <= PX * p):
                count += 1
                
    logo_f = np.copy(logo_sc).flatten() # logo flattened
    (ll,) = logo_f.shape
    
    logo_s = np.zeros((count,)) - 1 # logo scrambled
    logo_s[0:ll] = logo_f
    logo_s = scramble1D(logo_s)
    
    container = np.zeros((p2,p2))
    containerFilter = np.zeros((p2, p2))
    index = 0
    for n in range(1 + SN, p):
        for m in range(p2):
            l = m - p
            if(l % 4 != 0 and np.abs(l) + n >= DN and np.abs(l) + n <= PX * p):
                container[p + n, m] = logo_s[index]
                containerFilter[p + n, m] = 1
                index += 1
    
    containerFilter[np.where(container == -1)] = 0
    container[np.where(container == -1)] = 0
    
    container = container + np.rot90(np.rot90(container))
    containerFilter = containerFilter + np.rot90(np.rot90(containerFilter))
    
    return (container, containerFilter, index)

def watermarkExtract(momen_bw, p, ll):
    p2 = 2 * p + 1
    logo_f = np.array([])
    for n in range(1 + SN, p):
        for m in range(p2):
            l = m - p
            if(l % 4 != 0 and np.abs(l) + n >= DN and np.abs(l) + n <= PX * p):
                val = momen_bw[p + n, m]
                logo_f = np.append(logo_f, val)
    
    logo_f = unscramble1D(logo_f)
    logo_f = logo_f[0:ll*ll]
    logo_f = logo_f.reshape((ll,ll))
    return unscramble(logo_f)

In [2]:
# Attacks

# Rotation
def attackRotation(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# Resize
def attackResize(image, size):
    result = cv2.resize(image, (size, size))
    return result

# Translation
def attackTranslate(image, px, py):
    (h, w, c) = image.shape
    dx = int(px * w)
    dy = int(py * h)
#     result = np.random.randint(255, size=(h, w, c)).astype(np.uint8)
    result = np.zeros((h, w, c)).astype(np.uint8)
    if(dx > 0):
        result[:,dx:w,:] = image[:,0:w-dx,:]
    else:
        result[:,0:w+dx,:] = image[:,-dx:w,:]
#     result2 = np.random.randint(255, size=(h, w, c)).astype(np.uint8)
    result2 = np.zeros((h, w, c)).astype(np.uint8)
    if(dy > 0):
        result2[dy:h,:,:] = result[0:h-dy,:,:]
    else:
        result2[0:h+dy,:,:] = result[-dy:h,:,:]
    return result2

# Translation
def attackTranslateByPixel(image, dx, dy):
    (h, w, c) = image.shape
#     result = np.random.randint(255, size=(h, w, c)).astype(np.uint8)
    result = np.zeros((h, w, c)).astype(np.uint8)
    if(dx > 0):
        result[:,dx:w,:] = image[:,0:w-dx,:]
    else:
        result[:,0:w+dx,:] = image[:,-dx:w,:]
#     result2 = np.random.randint(255, size=(h, w, c)).astype(np.uint8)
    result2 = np.zeros((h, w, c)).astype(np.uint8)
    if(dy > 0):
        result2[dy:h,:,:] = result[0:h-dy,:,:]
    else:
        result2[0:h+dy,:,:] = result[-dy:h,:,:]
    return result2

# Crop
def attackCrop(image, tp, rp, bp, lp):
    (h, w, c) = image.shape
    dt = int(h*tp)
    db = int(h*bp)
    dl = int(w*lp)
    dr = int(w*rp)
    result = np.copy(image)
#     result[0:dt,:,:] = np.random.randint(255, size=(dt, w, c)).astype(np.uint8)
#     result[h-db:h,:,:] = np.random.randint(255, size=(db, w, c)).astype(np.uint8)
#     result[:,0:dl,:] = np.random.randint(255, size=(h, dl, c)).astype(np.uint8)
#     result[:,w-dl:w,:] = np.random.randint(255, size=(h, dr, c)).astype(np.uint8)
    result[0:dt,:,:] = np.zeros((dt, w, c)).astype(np.uint8)
    result[h-db:h,:,:] = np.zeros((db, w, c)).astype(np.uint8)
    result[:,0:dl,:] = np.zeros((h, dl, c)).astype(np.uint8)
    result[:,w-dl:w,:] = np.zeros((h, dr, c)).astype(np.uint8)
    return result

# Blur
def attackBlur(image, kernelSize):
    result = cv2.blur(image, (kernelSize, kernelSize))
    return result

# Noise
def attackRandomNoise(image, n_p):
    result = np.copy(image)
    for i, row in enumerate(image):
        for j, val in enumerate(row):
            if(np.random.randint(low=0, high=100) < n_p * 100 / 2):
                result[i,j] = 0
    for i, row in enumerate(image):
        for j, val in enumerate(row):
            if(np.random.randint(low=0, high=100) < n_p * 100 / 2):
                result[i,j] = 255
    return result

# JPEG Compression
def attackJPG(image, Q=95):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), Q]
    result, encimg = cv2.imencode('.jpg', image, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg

In [1]:
def processAttack(imgY, logo, p, g, delta):
    
    (h, w) = imgY.shape
    
    # Calculate Momen
    momen = transformPHT(imgY, h, p)
    
    # Current Binary Value & Difference
    matQ = np.floor(np.abs(momen) / delta) % 2
    
    # Crop Watermark
    extractedLogo = watermarkExtract(matQ, p, g) # MAT_Q_MODIFIED[logo_position]
    
    # Calculate Bit Error Rate    
    be = calcBE(logo, extractedLogo)
    ber = be / (g * g)
    
    return (extractedLogo, be, ber)

def displayAttackResult(image, attackResultTupple, attackName='', message=''):
    
    (logo, be, ber) = attackResultTupple
    
    # Show Container
    imgshow([[image, logo]], [[attackName, 'Extracted Watermark']])

    # Show Bit Error Rate
    print('BE  : ', be)
    print('BER : ', '{:.2%}'.format(ber))
    if(message):
        print(message)
    
def processAndDisplayAttack(img, ori_img, title, logo, p, g, delta, isNeedAntiTranslation):
    message = ''
    used_img = img
    
    if(isNeedAntiTranslation):
        (h, w, c) = img.shape
        cx = (h - 1) / 2
        (tx, ty) = correctTranslate(ori_img, img, (cx, cx))
        message = 'Inverse Translation X={} Y={}'.format(tx, ty)
        used_img = attackTranslateByPixel(img, tx, ty)
    
    imgY = getYFromBGR(used_img)
    
    result = processAttack(imgY, logo, p, g, delta)
    displayAttackResult(img, result, title, message)